In [18]:
import pandas as pd
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots
import colorsys
from community import community_louvain
import json
from datetime import datetime

In [5]:
df = pd.read_csv('/Users/dayanas/Library/CloudStorage/OneDrive-LIRNEasia/DAP/Code/Projects/Specters-of-conflict/build2.0/data/2022-01-01-2023-01-01-Sri_Lanka.csv')

In [6]:
df.head()

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,SRI6548,01 January 2023,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Sri Lanka),Tamil Ethnic Group (Sri Lanka),6,...,Vavuniya,8.7514,80.4971,1,Virakesari,National,"On 1 January 2023, Tamil relatives of the disa...",0,crowd size=no report,1680035158
1,SRI6584,01 January 2023,2023,1,Political violence,Riots,Mob violence,Rioters (Sri Lanka),NaN,5,...,Badulla,6.9895,81.0557,1,Thinakkural,National,"On 1 January 2023, a group of people attacked ...",0,crowd size=no report,1680035158
2,SRI6544,31 December 2022,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Sri Lanka),NaN,6,...,Poonakari,9.5052,80.2148,2,Virakesari,National,"On 31 December 2022, people staged a protest m...",0,crowd size=no report,1680035158
3,SRI6543,30 December 2022,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Sri Lanka),Hindu Group (Sri Lanka),6,...,Jaffna,9.6685,80.0074,1,Virakesari,National,"On 30 December 2022, a Hindu religious group p...",0,crowd size=no report,1680035158
4,SRI6547,30 December 2022,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Sri Lanka),Government of Sri Lanka (2019-),6,...,Tellippalai,9.7875,80.0304,1,Tamil Guardian; Virakesari,National-International,"On 30 December 2022, the Grama Niladhari offic...",0,crowd size=no report,1680035158


In [7]:
df.keys()

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp'],
      dtype='object')

In [8]:

actor1a = df['actor1'].to_list()
actor1b = df['assoc_actor_1'].to_list()
actor2a = df['actor2'].to_list()
actor2b = df['assoc_actor_2'].to_list()

In [9]:
import pandas as pd
import numpy as np
import networkx as nx

def create_graph_based_adjacency_matrix(actor1a, actor1b, actor2a, actor2b):
    # Create a graph
    G = nx.Graph()

    # Helper function to add edges for multiple actors
    def add_edges(actors1, actors2):
        if isinstance(actors1, str):
            actors1 = [actor.strip() for actor in actors1.split(';')]
        elif pd.isna(actors1):
            actors1 = []
        if isinstance(actors2, str):
            actors2 = [actor.strip() for actor in actors2.split(';')]
        elif pd.isna(actors2):
            actors2 = []

        for a1 in actors1:
            for a2 in actors2:
                if a1 and a2:  # Ensure non-empty strings
                    G.add_edge(a1, a2)

    # Add edges based on the relationships
    for a1, b1, a2, b2 in zip(actor1a, actor1b, actor2a, actor2b):
        add_edges(a1, b1)
        add_edges(b1, a2)
        add_edges(a2, b2)

    # Get all unique actors
    all_actors = sorted(G.nodes())

    # Create the adjacency matrix
    adj_matrix = nx.adjacency_matrix(G, nodelist=all_actors).todense()

    # Convert to DataFrame for better visualization
    adj_df = pd.DataFrame(adj_matrix, index=all_actors, columns=all_actors)

    return adj_df, G



adj_matrix, graph = create_graph_based_adjacency_matrix(actor1a, actor1b, actor2a, actor2b)
print("Adjacency Matrix:")
print(adj_matrix)

print("\nGraph Information:")
print(f"Number of nodes: {graph.number_of_nodes()}")
print(f"Number of edges: {graph.number_of_edges()}")
print("Degree of each node:")
for node, degree in graph.degree():
    print(f"{node}: {degree}")

# Example of using graph theory concepts
print("\nGraph Theory Applications:")
print(f"Graph density: {nx.density(graph)}")
print(f"Is the graph connected? {nx.is_connected(graph)}")
if not nx.is_connected(graph):
    print(f"Number of connected components: {nx.number_connected_components(graph)}")
print(f"Average clustering coefficient: {nx.average_clustering(graph)}")

Adjacency Matrix:
                                         ACHWU: All Ceylon Health Workers' Union  \
ACHWU: All Ceylon Health Workers' Union                                        0   
ACMC: All Ceylon Makkal Congress                                               0   
Buddhist Group (Sri Lanka)                                                     0   
CPSL: Communist Party of Sri Lanka                                             0   
CTU: Ceylon Teachers Union                                                     0   
...                                                                          ...   
Teachers (Sri Lanka)                                                           0   
UNP: United National Party                                                     0   
UPFA: United People's Freedom Alliance                                         0   
UTUC: United Trade Union Confederation                                         0   
Women (Sri Lanka)                                         

In [10]:
adj_matrix

,ACHWU: All Ceylon Health Workers' Union,ACMC: All Ceylon Makkal Congress,Buddhist Group (Sri Lanka),CPSL: Communist Party of Sri Lanka,CTU: Ceylon Teachers Union,Catholic Christian Group (Sri Lanka),Christian Group (Sri Lanka),Civilians (Sri Lanka),DLF: Democratic Left Front,DPF: Democratic People's Front,...,TNPF: Tamil National People's Front,TPA: Tamil Progressive Alliance,TUCC: Trade Union Coordinating Centre,TUMO: Trade Unions and Mass Organisations,Tamil Ethnic Group (Sri Lanka),Teachers (Sri Lanka),UNP: United National Party,UPFA: United People's Freedom Alliance,UTUC: United Trade Union Confederation,Women (Sri Lanka)
ACHWU: All Ceylon Health Workers' Union,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACMC: All Ceylon Makkal Congress,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Buddhist Group (Sri Lanka),0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
CPSL: Communist Party of Sri Lanka,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CTU: Ceylon Teachers Union,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Teachers (Sri Lanka),0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
UNP: United National Party,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
UPFA: United People's Freedom Alliance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
UTUC: United Trade Union Confederation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
adj_matrix.index

Index(['ACHWU: All Ceylon Health Workers' Union',
       'ACMC: All Ceylon Makkal Congress', 'Buddhist Group (Sri Lanka)',
       'CPSL: Communist Party of Sri Lanka', 'CTU: Ceylon Teachers Union',
       'Catholic Christian Group (Sri Lanka)', 'Christian Group (Sri Lanka)',
       'Civilians (Sri Lanka)', 'DLF: Democratic Left Front',
       'DPF: Democratic People's Front', 'FLSP: Frontline Socialist Party',
       'Farmers (Sri Lanka)', 'Fishers (India)', 'Fishers (Sri Lanka)',
       'Former Government of Sri Lanka (2019-)',
       'GMOA: Government Medical Officers' Association',
       'Government of Sri Lanka (2019-)', 'Health Workers (Sri Lanka)',
       'Henawatta Communal Group (Sri Lanka)', 'Hindu Group (Sri Lanka)',
       'ITAK: Illankai Tamil Arasu Kachchi',
       'IUBF: Inter University Bhikkhu Front',
       'IUSF: Inter University Students' Federation',
       'IUTUF: Inter University Trade Union Federation',
       'JUGU: Joint Unemployed Graduates' Union',
       'J

In [12]:
type(adj_matrix)

pandas.core.frame.DataFrame

In [13]:
import networkx as nx

# Example adjacency matrix as a DataFrame
# Assume df is your DataFrame


df = pd.DataFrame(adj_matrix, index=adj_matrix.index)

# Create a graph from the DataFrame
G = nx.from_pandas_adjacency(df)

# Verify the nodes and edges
print("Nodes:", G.nodes())
print("Edges:", G.edges())

# If the adjacency matrix has weights, use this code instead:
# G = nx.from_pandas_adjacency(df, create_using=nx.Graph(), edge_attr='weight')


Nodes: ["ACHWU: All Ceylon Health Workers' Union", 'ACMC: All Ceylon Makkal Congress', 'Buddhist Group (Sri Lanka)', 'CPSL: Communist Party of Sri Lanka', 'CTU: Ceylon Teachers Union', 'Catholic Christian Group (Sri Lanka)', 'Christian Group (Sri Lanka)', 'Civilians (Sri Lanka)', 'DLF: Democratic Left Front', "DPF: Democratic People's Front", 'FLSP: Frontline Socialist Party', 'Farmers (Sri Lanka)', 'Fishers (India)', 'Fishers (Sri Lanka)', 'Former Government of Sri Lanka (2019-)', "GMOA: Government Medical Officers' Association", 'Government of Sri Lanka (2019-)', 'Health Workers (Sri Lanka)', 'Henawatta Communal Group (Sri Lanka)', 'Hindu Group (Sri Lanka)', 'ITAK: Illankai Tamil Arasu Kachchi', 'IUBF: Inter University Bhikkhu Front', "IUSF: Inter University Students' Federation", 'IUTUF: Inter University Trade Union Federation', "JUGU: Joint Unemployed Graduates' Union", 'JVP: Janatha Vimukthi Peramuna', 'Journalists (Sri Lanka)', 'LGBTQ+ (Sri Lanka)', 'LSSP: Lanka Sama Samaja Party

In [14]:
import numpy as np
from scipy.linalg import eigh

# Adjacency matrix
A = nx.to_numpy_array(G)

# Degree matrix
D = np.diag(A.sum(axis=1))

# Laplacian matrix
L = D - A

# Eigenvalues and eigenvectors
eigenvalues, eigenvectors = eigh(L)

print("Eigenvalues of the Laplacian matrix:")
print(eigenvalues)


Eigenvalues of the Laplacian matrix:
[3.08148016e-15 3.70010897e-01 5.65513739e-01 8.66349401e-01
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.05677528e+00 1.37310874e+00 1.65232893e+00 1.75467514e+00
 1.82342806e+00 1.98478113e+00 2.00000000e+00 2.00000000e+00
 2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00
 2.00000000e+00 2.00000000e+00 2.00000000e+00 2.36961665e+00
 2.61622656e+00 2.84038721e+00 2.87981113e+00 2.96710409e+00
 3.00000000e+00 3.00000000e+00 3.00000000e+00 3.00000000e+00
 3.35548417e+00 3.53198940e+00 4.00000000e+00 4.00000000e+00
 4.00000000e+00 4.00000000e+00 4.24457290e+00 5.

In [15]:
# PageRank
pagerank = nx.pagerank(G)
print("PageRank centrality metrics:")
print(pagerank)


PageRank centrality metrics:
{"ACHWU: All Ceylon Health Workers' Union": 0.005599137268794836, 'ACMC: All Ceylon Makkal Congress': 0.005599137268794836, 'Buddhist Group (Sri Lanka)': 0.014054991156971789, 'CPSL: Communist Party of Sri Lanka': 0.005599137268794836, 'CTU: Ceylon Teachers Union': 0.00837491624435047, 'Catholic Christian Group (Sri Lanka)': 0.008414632670749024, 'Christian Group (Sri Lanka)': 0.005599137268794836, 'Civilians (Sri Lanka)': 0.0522521465521209, 'DLF: Democratic Left Front': 0.005599137268794836, "DPF: Democratic People's Front": 0.005599137268794836, 'FLSP: Frontline Socialist Party': 0.008463716779461969, 'Farmers (Sri Lanka)': 0.008414632670749024, 'Fishers (India)': 0.010780838007600928, 'Fishers (Sri Lanka)': 0.014054991156971789, 'Former Government of Sri Lanka (2019-)': 0.014054991156971789, "GMOA: Government Medical Officers' Association": 0.005599137268794836, 'Government of Sri Lanka (2019-)': 0.01758290308644525, 'Health Workers (Sri Lanka)': 0.0111

In [16]:

# Perform Spectral Analysis on G
A = nx.to_numpy_array(G)
D = np.diag(A.sum(axis=1))
L = D - A

# Eigenvalues and eigenvectors
eigenvalues, eigenvectors = eigh(L)

# Spectral Clustering
k = 3  # Number of clusters
kmeans = KMeans(n_clusters=k)
kmeans.fit(eigenvectors[:, 1:k+1])
labels = kmeans.labels_

# Louvain Community Detection
partition = community_louvain.best_partition(G)

# PageRank Centrality
pagerank = nx.pagerank(G)

# Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(G)

# Combine metrics for node importance
node_importance = {node: (eigenvector_centrality[node] + pagerank[node]) / 2 for node in G.nodes()}

# 3D spring layout
pos = nx.spring_layout(G, dim=3, k=2, iterations=100)

# Extract and normalize node positions
x_nodes, y_nodes, z_nodes = zip(*[pos[node] for node in G.nodes()])
x_nodes = np.interp(x_nodes, (min(x_nodes), max(x_nodes)), (0, 100))
y_nodes = np.interp(y_nodes, (min(y_nodes), max(y_nodes)), (0, 100))
z_nodes = np.interp(z_nodes, (min(z_nodes), max(z_nodes)), (0, 100))

# Extract edges
edges = G.edges(data=True)
x_edges, y_edges, z_edges, weights = [], [], [], []
for edge in edges:
    x_edges.extend([pos[edge[0]][0], pos[edge[1]][0], None])
    y_edges.extend([pos[edge[0]][1], pos[edge[1]][1], None])
    z_edges.extend([pos[edge[0]][2], pos[edge[1]][2], None])
    weights.append(edge[2]['weight'])

# Function to safely normalize edge positions
def safe_normalize(edges):
    non_none = [e for e in edges if e is not None]
    if not non_none:  # If all values are None
        return edges
    min_val, max_val = min(non_none), max(non_none)
    if min_val == max_val:  # If all non-None values are the same
        return [100 if e is not None else None for e in edges]
    return [np.interp(e, (min_val, max_val), (0, 100)) if e is not None else None for e in edges]

# Normalize edge positions and weights
x_edges = safe_normalize(x_edges)
y_edges = safe_normalize(y_edges)
z_edges = safe_normalize(z_edges)
weights_normalized = np.interp(weights, (min(weights), max(weights)), (1, 10))

# Generate pastel colors for nodes
def generate_pastel_color(h):
    return "#{:02x}{:02x}{:02x}".format(*[int(x * 255) for x in colorsys.hsv_to_rgb(h, 0.5, 0.95)])

node_colors = [generate_pastel_color(h) for h in np.linspace(0, 1, len(G.nodes()), endpoint=False)]

# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'xy'}]],
                    subplot_titles=("Graph Visualization", "Graph Laplacian"))

# Add edges to the first subplot
for i in range(len(weights_normalized)):
    fig.add_trace(go.Scatter3d(x=x_edges[i*3:(i+1)*3], y=y_edges[i*3:(i+1)*3], z=z_edges[i*3:(i+1)*3],
                               mode='lines',
                               line=dict(color='rgba(200,200,200,0.5)', width=weights_normalized[i]),
                               hoverinfo='none'),
                  row=1, col=1)

# Add nodes to the first subplot
node_sizes = [30 + 100 * node_importance[node] for node in G.nodes()]
fig.add_trace(go.Scatter3d(x=x_nodes, y=y_nodes, z=z_nodes,
                           mode='markers+text',
                           marker=dict(symbol='circle', 
                                       size=node_sizes, 
                                       color=[partition[node] for node in G.nodes()],
                                       colorscale='Viridis', 
                                       line=dict(color='black', width=1)),
                           text=[f'Node {node}' for node in G.nodes()],
                           textposition='top center',
                           hoverinfo='text'),
              row=1, col=1)

# Add Graph Laplacian to the second subplot
heatmap = go.Heatmap(z=L, colorscale='Hot', zmin=np.min(L), zmax=np.max(L))
fig.add_trace(heatmap, row=1, col=2)

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(showbackground=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showticklabels=False, title=''),
    ),
    xaxis2=dict(title='Node'),
    yaxis2=dict(title='Node'),
    showlegend=False,
    title="Graph Analysis and Visualization",
    height=600,
    width=1200,
)

# Show the figure
fig.show()

# Print analysis results
print("Eigenvalues of the Laplacian matrix:")
print(eigenvalues)
print("\nPageRank Centrality:")
print(pagerank)
print("\nEigenvector Centrality:")
print(eigenvector_centrality)
print("\nLouvain Communities:")
print(partition)

Eigenvalues of the Laplacian matrix:
[3.08148016e-15 3.70010897e-01 5.65513739e-01 8.66349401e-01
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.05677528e+00 1.37310874e+00 1.65232893e+00 1.75467514e+00
 1.82342806e+00 1.98478113e+00 2.00000000e+00 2.00000000e+00
 2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00
 2.00000000e+00 2.00000000e+00 2.00000000e+00 2.36961665e+00
 2.61622656e+00 2.84038721e+00 2.87981113e+00 2.96710409e+00
 3.00000000e+00 3.00000000e+00 3.00000000e+00 3.00000000e+00
 3.35548417e+00 3.53198940e+00 4.00000000e+00 4.00000000e+00
 4.00000000e+00 4.00000000e+00 4.24457290e+00 5.

In [19]:


def dump_params_to_json(params):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    for idx, param in enumerate(params):
        if isinstance(param, np.ndarray):
            filename = f'array_{idx}_{timestamp}.json'
            data = param.tolist()  # Convert numpy array to list
        elif isinstance(param, dict):
            filename = f'dict_{idx}_{timestamp}.json'
            data = param
        else:
            raise TypeError(f"Parameter at index {idx} is neither a numpy array nor a dictionary.")
        
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
        
        print(f"Dumped parameter {idx} to {filename}")

params = [eigenvalues,pagerank,eigenvector_centrality,partition]

dump_params_to_json(params)


Dumped parameter 0 to array_0_2024-06-28_01-38-39.json
Dumped parameter 1 to dict_1_2024-06-28_01-38-39.json
Dumped parameter 2 to dict_2_2024-06-28_01-38-39.json
Dumped parameter 3 to dict_3_2024-06-28_01-38-39.json
